# Rectifying imbalanced features with Synthetic methods and trying some models

In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter
import pandas as pd
import numpy as np
import sklearn

In [ ]:
test = pd.read_csv("../input/features-corected/Feature_Corrected_Test.csv")
submit =  pd.read_csv("../input/av-healthcare-analytics-ii/healthcare/sample_sub.csv")
train = pd.read_csv("../input/features-corected/Feature_Corrected_Train.csv")
train.pivot_table(index=["Stay"],aggfunc= "size")

In [ ]:
y = train.Stay
X = train.drop("Stay",axis = 1)
X

In [ ]:
smote = SMOTE({0:50000,4:50000,5:50000,6:20000,7:20000,8:20000,9:20000,10:20000},random_state= 0)
X_res,y_res = smote.fit_resample(X,y)
print('Resampled dataset shape %s' % Counter(y_res))

# Here is the hyperparameters that I tuned change the method to gpu_hist if you are using a GPU

In [ ]:
boost = "gbdt"
lr = 0.21
trees = 300
max_depth = 10
crit = "gini"
split = 4
leaf = 4
leaf_weight = 0.2
features_select = "auto"
random_state = 0
class_weight = None
obj = "multiclass"
reg_l = 1.1
reg_al = 1.6
# method = "gpu_hist" Uncomment if using a GPU

In [ ]:
from sklearn.model_selection import train_test_split
train_X,valid_X,train_y,valid_y = train_test_split(X_res,y_res,test_size = 0.2,stratify = y_res,random_state = 0)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=800,criterion=crit,min_samples_split=split,
                           min_samples_leaf=leaf,min_weight_fraction_leaf= leaf_weight,max_features=features_select,
                           random_state=random_state, class_weight=class_weight,n_jobs=-1)
rf.fit(train_X,train_y)
rf.score(valid_X,valid_y)

In [ ]:
import lightgbm
lg = lightgbm.LGBMClassifier(boosting_type=boost,max_depth=max_depth,learning_rate=lr,n_estimators=trees,
                            objective=obj,class_weight=class_weight,reg_alpha=reg_al,reg_lambda=reg_l,
                            random_state=random_state,n_jobs=-1)
lg.fit(train_X,train_y)
lg.score(valid_X,valid_y)

In [ ]:
import xgboost
xg = xgboost.XGBClassifier(max_depth= max_depth,learning_rate = lr,objective="multi:softmax",booster = "gbtree",
                           min_child_weight = leaf_weight,reg_alpha = reg_al,reg_lambda = reg_l,
                          n_jobs = -1,random_state = random_state,num_classes = 11,eval_metric ="logloss")
                          #tree_method = method)
xg.fit(train_X,train_y)
xg.score(valid_X,valid_y)

In [ ]:
import catboost
ct = catboost.CatBoostClassifier(iterations= 100,learning_rate=lr,depth=max_depth,loss_function="Logloss",
                                reg_lambda=reg_l,class_weights=class_weight,objective="MultiClass",
                                random_state=random_state) #task_type = "GPU")

ct.fit(train_X,train_y)
ct.score(valid_X,valid_y)